<H3>PRI 2023/24: first project delivery</H3>

**GROUP X**
- name, number
- name, number
- name, number

<H3>Part I: demo of facilities</H3>

In [1]:
import os

In [2]:
def read_files(path):
    texts = []
    for folder in os.listdir(path):
        category_path = os.path.join(path, folder)
        texts.append([])
        for file in os.listdir(category_path):
            file_path = os.path.join(category_path, file)
            with open(file_path, "r") as f:
                text = f.read()
                texts[-1].append(text)
                
    print("Number of Categories:",len(os.listdir(path)))
    for i in range(len(os.listdir(path))):
        print("Number of Articles in", "'"+os.listdir(path)[i]+"'", "Category:",len(texts[i]))
    return texts

In [3]:
articles = read_files('BBC News Summary\\BBC News Summary\\News Articles')

Number of Categories: 5
Number of Articles in 'business' Category: 510
Number of Articles in 'entertainment' Category: 386
Number of Articles in 'politics' Category: 417
Number of Articles in 'sport' Category: 511
Number of Articles in 'tech' Category: 401


In [4]:
#Examplary text. The structure of the read file is: articles[category_no][document_no]. 
print(articles[0][0])

Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AO

A) **Indexing** (preprocessing and indexing options)

In [ ]:
#code, statistics and/or charts here

B) **Summarization**

*B.1 Summarization solution: results for a given document*

In [ ]:
#code, statistics and/or charts here

*B.2 IR models (TF-IDF, BM25 and EBRT)*

In [ ]:
#code, statistics and/or charts here

*B.3 Reciprocal rank funsion*

In [ ]:
#code, statistics and/or charts here

*B.4 Maximal Marginal Relevance*

In [ ]:
#code, statistics and/or charts here

C) **Keyword extraction**

In [ ]:
#code, statistics and/or charts here

D) **Evaluation**

In [ ]:
#code, statistics and/or charts here

<H3>Part II: questions materials (optional)</H3>

**(1)** Corpus *D* and summaries *S* description.

In [ ]:
#code, statistics and/or charts here

**(2)** Summarization performance for the overall and category-conditional corpora.

In [ ]:
#code, statistics and/or charts here

**...** (additional questions with empirical results)

<H3>END</H3>